# 0.0 IMPORTS

    Bibliotecas utilizadas


In [190]:
import os
import math
import numpy                as np
import pickle
import pandas               as pd
import xgboost              as xgb
import seaborn              as sns
import inflection
import matplotlib.pyplot    as plt

from scipy                      import stats    as ss
from boruta                     import BorutaPy
from datetime                   import datetime, timedelta
from IPython.display            import Image
from sklearn.metrics            import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.ensemble           import RandomForestRegressor
from sklearn.linear_model       import LinearRegression, Lasso
from sklearn.preprocessing      import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection    import train_test_split

## 0.1 FUNCTIONS
    
    Funções utilizadas

In [191]:
def is_promo(row):
    if row['promo_interval'] == 0:
        return 0
    elif row['month_map'] in row['promo_interval'].split(','):
        return 1
    else:
        return 0

def cramer_v (x,y):

    cm = pd.crosstab(x, y).to_numpy()
    n=cm.sum()
    r,k=cm.shape
    
    chi2 = ss.chi2_contingency(cm)[0]
    
    chi2corr = max(0, chi2 - (k-1)*(r-1)/(n-1))
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    
    v = np.sqrt((chi2corr/n)/(min(kcorr-1,rcorr-1)))
    
    return v

def ml_error(model_name, y, yhat):
    mae = mean_absolute_error(y,yhat)
    mape = mean_absolute_percentage_error(y,yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    
    return pd.DataFrame({'Model Name': model_name,
                         'MAE': mae,
                         'MAPE': mape,
                         'RMSE':rmse }, index=[0])

def cross_validation(x_training, kfold, model_name, model, verbose=False):
    mae_list=[]
    mape_list=[]
    rmse_list=[]

    for k in reversed(range(1,kfold+1)):
        if verbose:
            print('\nKFold Number: {}'.format(k))
        #start and end date for validation
        validation_start_date = x_training['date'].max()-timedelta(days=6*7*(k))
        validation_end_date = x_training['date'].max()-timedelta(days=6*7*(k-1))

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]


        #
        xtraining = training.drop(['date', 'sales'], axis=1)
        ytraining = training['sales']

        xvalidation = validation.drop(['date', 'sales'], axis=1)
        yvalidation = validation['sales']


        m = model.fit(xtraining, ytraining)

        yhat = m.predict(xvalidation)

        m_result = ml_error(model_name, yvalidation, yhat)
        mae_list.append(m_result['MAE'])
        mape_list.append(m_result['MAPE'])
        rmse_list.append(m_result['RMSE'])
        
    return pd.DataFrame({'Model Name': model_name,
                         'MAE CV':  np.round(np.mean(mae_list),2).astype(str) + '+/-' + np.round(np.std(mae_list),2).astype(str),
                         'MAPE CV': np.round(np.mean(mape_list),2).astype(str) + '+/-' + np.round(np.std(mape_list),2).astype(str),
                         'RMSE CV': np.round(np.mean(rmse_list),2).astype(str) + '+/-' + np.round(np.std(rmse_list),2).astype(str)}, index=[0])

## 0.2 DIRETÓRIOS/LOADING DATA

    Definição dos diretórios e leitura dos dados.


In [192]:
BASE_DIR = os.path.abspath('')   
DATA_DIR = os.path.join(BASE_DIR,'data')
PARA_DIR = os.path.join(BASE_DIR,'parameter')

df_sales_raw = pd.read_csv(os.path.join(DATA_DIR,'train.csv'), low_memory = False)
df_store_raw = pd.read_csv(os.path.join(DATA_DIR,'store.csv'), low_memory = False)
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')
df1 = df_raw.copy()

# 1.0 DATA DISCRIPTION

    Verificações e alterações básicas no dataframe



## 1.1 Rename Columns
    As colunas são reescritas no modo snake, todas minusculas e com _ no lugar de espaços

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
            'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2 Data Dimensions / Types
    Dimensões do dataframe de treino e os tipos de colunas

In [ ]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))
df1.dtypes

## 1.3 Check // Fillout NA
    Verificação das colunas e quantidade de linhas que possuem NaN. Esses valores faltantes são preenchidos com base na lógica do negócio

In [ ]:
print(df1.isna().sum())

#### Preenchimento dos valores NaN
###### Todas presunções assumidass são baseadas nos meus conhecimentos em problemas de negócio e não necessáriamente refletem a realidade das lojas Rossmann

##### competition_distance
    Os valores não preenchidos nessa coluna provavelmte significam que essas lojas não possuem um competidor próximo, portanto, para preencher essas coluna utilizarei valores de distancias muito grandes, no caso 200000 metros.

##### competition_open_since_month
    Aqui temos o tempo, por mês, em que a ultima loja concorrente abriu. Os valores NaN se devem provavelmente por lojas não possuem competidor perto, ou que o competidor já existia no momento em que a loja da Rossmann foi inaugurada, ou ainda por simplesmente por falta de acesso a essa informação. Vamos considerar então a data da venda como essa data, dessa forma a diferença entre a data de venda e abertura será zero para os valores NaN.

##### competition_open_since_year // promo2_since_week // promo2_since_year
    Aplicaremos a mesma lógica do caso anterior

##### promo_interval
    0 se não houve promoção

Além disso também foi alterado os meses de números para seus nomes abreviados.



In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
# competition_distance
df1['competition_distance'].fillna(200000.0, inplace=True)

# competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

# competition_open_since_year
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

# promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

# promo_interval
df1['promo_interval'].fillna(0, inplace=True)


month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
df1['month_map'] = df1['date'].dt.month.map(month_map)
df1['is_promo'] = df1.apply(is_promo, axis=1)

## 1.5 Change Types
    Transformação nos tipos de variaveis das colunas, se necessário

In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

## 1.6 Descriptive Statistical
    Analise estatistica descritiva das variaveis. Para as variaveis numericas foram tomadas medidas de tendencia central (média e mediana) e medidas de dispersão (desviopadrão, range, skewness e kurtosis). Para as váriaveis categóricas foi realizado um boxplot.

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

In [ ]:
#Central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max()-x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()

m.columns = (['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis'])

m


In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

fig = plt.subplots(1, 3, figsize=(21, 7))

plt.subplot(1,3,1)
sns.boxplot(x='state_holiday', y='sales', data=aux1)

plt.subplot(1,3,2)
sns.boxplot(x='store_type', y='sales', data=aux1)

plt.subplot(1,3,3)
sns.boxplot(x='assortment', y='sales', data=aux1)

# 2.0 FEATURE ENGINEERING


In [ ]:
df2 = df1.copy()

## 2.1 Mapa mental das hipoteses
    Para dar inicio ao processo de feature engineering foi criado um mapa mental de hipoteses, onde, me baseando no problema de negócio estabelecido, criei diferentes hipoteses separadas por Lojas, Clientes, Produtos, Localização e Sazonalidade.

In [ ]:
Image('images/mind_map_hipoteses.png')

## 2.2 Criação das hipoteses


### hipoteses  loja

**1.** Lojas com maior quadro de funcionarios deveriam vender mais;

**2.** Lojas com maior estoque deveriam vender mais;

**3.** Lojas com maior porte deveriam vender mais;

**4.** Lojas com menor porte deveriam vender menos;

**5.** Lojas com maior sortimento deveriam vender mais;

**6.** Lojas com competidores mais próximos deveriam vender menos;

**7.** Lojas com competidores à mais tempo deveriam vender mais.

### hipoteses do produto

**1.** Lojas que investem mais em marketing deveriam vender mais;

**2.** Lojas que expoem mais os produtos na vitrine deveriam vender mais;

**3.** Lojas que tem preços menores nos produtos deveriam vender mais;

**4.** Lojas que tem preços menores por mais tempo nos produtos deveriam vender mais.

**5.** Lojas com promoções mais agressivais deveriam vender mais;

**6.** Lojas com mais dias de promoção deveriam vender mais;

**7.** Lojas com mais promoções consecutivas deveriam vender mais.

### hipoteses de tempo

**1.** Lojas que abrem durante o natal deveriam vender mais;

**2.** Lojas deveriam vender mais ao longo dos anos;

**3.** Lojas deveriam vender mais no segundo semestre do ano;

**4.** Lojas deveria vender mais após o dia 10 de cada mês;

**5.** Lojas deveriam vender menos aos finais de semana;

**6.** Lojas deveriam vender menos durante feriados escolares;

### Lista final de hipoteses

    Nem todas hipóteses anteriores podem ser respondidas com o dataset disponibilizado, devido a isso seleceionei apenas algumas:

**1.** Lojas com maior sortimento deveriam vender mais;

**2.** Lojas com competidores mais próximos deveriam vender menos;

**3.** Lojas com competidores à mais tempo deveriam vender mais.

--------------------------------------------------------------------------------

**4.** Lojas que tem preços menores por mais tempo nos produtos deveriam vender mais.

**5.** Lojas com mais dias de promoção deveriam vender mais;

**6.** Lojas com mais promoções consecutivas deveriam vender mais.

--------------------------------------------------------------------------------

**7.** Lojas que abrem durante o natal deveriam vender mais;

**8.** Lojas deveriam vender mais ao longo dos anos;

**9.** Lojas deveriam vender mais no segundo semestre do ano;

**10.** Lojas deveria vender mais após o dia 10 de cada mês;

**11.** Lojas deveriam vender menos aos finais de semana;

**12.** Lojas deveriam vender menos durante feriados escolares;

## 2.3 Feature Engineering
    A partir de agora começa o processo de extração de novas variavéis a partir das disponiveis no dataframe. As variaveis derivadas abaixo são, respectivamente:
    - Ano
    - Mês
    - Dia
    - Semana do ano
    - Ano-Semana
    - Tempo de competição
    - Tempo de competição por mês
    - Tempo de promoção
    - Tempo de promoção por semana
    - Alteração nos valores de assortment/state_holiday para seus devidos significados

In [ ]:
#Year
df2['year'] = df2['date'].dt.year

#Month
df2['month'] = df2['date'].dt.month

#Day
df2['day'] = df2['date'].dt.day

#Week of Year
df2['week_of_year'] = df2['date'].dt.isocalendar().week

#Year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')


#competition since
df2['competition_since'] = df2.apply(lambda x: datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1),axis=1)
df2['competition_time_month'] = ((df2['date']-df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

#Promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str)+'-'+df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.strptime(x+'-1','%Y-%W-%w')-timedelta(days=7))
df2['promo_time_week'] = ((df2['date']-df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

#assortment
df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x=='a' else 'extra' if x=='b' else 'extended')

#state holiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x=='a' else 'easter_holiday' if x=='b' else 'christmas' if x == 'c' else 'regular_day')

# 3.0 FILTRAGEM DE VARIAVEIS

In [ ]:
df3 = df2.copy()

## 3.1 Filtragem das linhas
    Linhas onde a loja estava fechada ou com vendas iguais a zero não serão úteis para a modelagem. Tiramos elas a seguir.

In [ ]:
df3 = df3[df3['open'] != 0]
df3 = df3[df3['sales'] > 0]

## 3.2 Seleção das colunas
    Agora também selecionamos as colunas que não agregam informação para o modelo.

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis=1)

# 4.0 ANALISE EXPLORATORIA DOS DADOS
    Inicio da analise exploratória. Separamos novamente as variaveis em categoricas e numericas

In [ ]:
df4 = df3.copy()
num_attributes = df4.select_dtypes(include=['int64', 'float64'])
cat_attributes = df4.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

## 4.1 Analise univariada
    Analise univariada nas variaveis resposta, númericas e categóricas

### 4.1.1 Variavel resposta

In [ ]:
#Variavel resposta
sns.histplot(df4['sales'], kde=True, stat="density");

### 4.1.2Variaveis númericas

In [ ]:
num_attributes.hist(bins=25, figsize=(21,14));

In [ ]:
#Variaveis categoricas
df4['state_holiday'].drop_duplicates()

In [ ]:
#state holiday
a=df4[df4['state_holiday'] != 'regular_day']
fig = plt.subplots(3,2,figsize=(21,21))
plt.subplot(3,2,1)
sns.countplot(a,x='state_holiday')

plt.subplot(3,2,2)
sns.kdeplot(df4[df4['state_holiday'] == 'public_holiday']['sales'], label='public_holiday', fill=True);
sns.kdeplot(df4[df4['state_holiday'] == 'easter_holiday']['sales'], label='public_holiday', fill=True);
sns.kdeplot(df4[df4['state_holiday'] == 'christmas']['sales'], label='public_holiday', fill=True);

#store_type
plt.subplot(3,2,3)
sns.countplot(df4,x='store_type')

plt.subplot(3,2,4)
sns.kdeplot(df4[df4['store_type'] == 'a']['sales'], label='a', fill=True);
sns.kdeplot(df4[df4['store_type'] == 'b']['sales'], label='b', fill=True);
sns.kdeplot(df4[df4['store_type'] == 'c']['sales'], label='c', fill=True);
sns.kdeplot(df4[df4['store_type'] == 'd']['sales'], label='d', fill=True);

#assortment
plt.subplot(3,2,5)
sns.countplot(df4,x='assortment')

plt.subplot(3,2,6)
sns.kdeplot(df4[df4['assortment'] == 'extended']['sales'], label='extended', fill=True);
sns.kdeplot(df4[df4['assortment'] == 'basic']['sales'], label='basic', fill=True);
sns.kdeplot(df4[df4['assortment'] == 'extra']['sales'], label='extra', fill=True);


## 4.2 Analise bivariada
    Aqui foi verificado se as hipoteses criadas anteriormente são verdadeiras ou falsas.

### H1 lojas com maior sortimentos deveriam vender mais

***FALSA*** Na verdade lojas com maior sortimento vende menos


In [ ]:
aux1 = df4[['assortment','sales']].groupby('assortment').sum().reset_index()

sns.barplot(x='assortment', y='sales', data=aux1);

aux2 = df4[['year_week','assortment','sales']].groupby(['year_week', 'assortment']).sum().reset_index()
aux2.pivot(index='year_week', columns='assortment', values='sales').plot()

aux3 = aux2[aux2['assortment'] =='extra']
aux3.pivot(index='year_week', columns='assortment', values='sales').plot()


### H2 lojas com competidores mais próximos deveriam vender menos
***FALSE*** Loja com competidores próximos vendem mais e não menos

In [ ]:
aux1 = df4[['competition_distance','sales']].groupby('competition_distance').sum().reset_index()

bins = list(np.arange(0,20000,1000))

fig = plt.subplots(figsize=(21,14));
plt.subplot(2,2,1);
sns.scatterplot(x='competition_distance', y='sales', data=aux1);

plt.subplot(2,2,2);
sns.heatmap(aux1.corr(method='pearson'), annot=True);

aux1['competition_distance_binned'] = pd.cut(aux1['competition_distance'], bins = bins)
aux2 = aux1[['competition_distance_binned','sales']].groupby('competition_distance_binned').sum().reset_index()


plt.subplot(2,2,(3,4));
sns.barplot(x='competition_distance_binned', y='sales', data=aux2);
plt.xticks(rotation=-45);



### H3 loja com competidores a mais tempo deveriam vender mais
***FALSO*** loja com competidores a mais tempo vendem menos

In [ ]:
plt.subplots(2,2,figsize=(21,14))

aux1 = df4[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()
aux2 = aux1[aux1['competition_time_month']<120]
aux2 = aux2[aux2['competition_time_month'] != 0]


plt.subplot(2,2,1)
sns.regplot(x='competition_time_month', y='sales', data=aux2);

plt.subplot(2,2,2)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

plt.subplot(2,2,(3,4))
sns.barplot(x='competition_time_month', y='sales', data=aux2);
plt.xticks(rotation=-45);


### H4 Lojas com promoções ativas por mais tempo deveriam vender mais.

***FALSO*** Ela vende regularmente por um periodo, porém tempo depois passa a cair

In [ ]:

aux1 = df4[['promo_time_week','sales']].groupby('promo_time_week').sum().reset_index()

aux2 = aux1[aux1['promo_time_week']>0] #periodo extendido
aux3 = aux1[aux1['promo_time_week']<0] #periodo regular

fig = plt.subplots(2,3,figsize=(21,14))
plt.subplot(2,3,1)
sns.barplot(x='promo_time_week', y='sales', data=aux2);
plt.xticks(rotation=-45);

plt.subplot(2,3,4)
sns.regplot(x='promo_time_week', y='sales', data=aux2);
plt.xticks(rotation=-45);

plt.subplot(2,3,2)
sns.barplot(x='promo_time_week', y='sales', data=aux3);
plt.xticks(rotation=-45);

plt.subplot(2,3,5)
sns.regplot(x='promo_time_week', y='sales', data=aux3);
plt.xticks(rotation=-45);

plt.subplot(2,3,(3,6))
sns.heatmap(aux1.corr(method='pearson'), annot=True)

### <s>H5 Lojas com mais dias de promoção deveriam vender mais</s>

### H6 Lojas com mais promoções consecutivas deveriam vender mais

***FALSO*** Lojas com mais promo consec, vendem menos

In [ ]:
df4[['promo','promo2','sales']].groupby(['promo', 'promo2']).sum().reset_index()


aux1 = df4[df4['promo'] == 1]
aux1 = aux1[aux1['promo2'] == 1][['year_week','sales']].groupby('year_week').sum().reset_index()
ax = aux1.plot()

aux2 = df4[df4['promo'] == 1]
aux2 = aux2[aux2['promo2'] == 0][['year_week','sales']].groupby('year_week').sum().reset_index()
aux2.plot(ax=ax)

ax.legend(labels=['Tradicional e extendida', 'Extendida'])

### H7 Lojas abertas durante o feriado de Natal deveriam vender mais

***FALSA*** Lojas abertas durante o feriado de natal vendem menos

In [ ]:
aux = df4[df4['state_holiday'] != 'regular_day']

fig=plt.subplots(1,2,figsize=(21,7))

plt.subplot(1,2,1)
aux1 = aux[['state_holiday','sales']].groupby('state_holiday').sum().reset_index()
sns.barplot(x='state_holiday', y='sales', data = aux1);

plt.subplot(1,2,2)
aux2 = aux[['year', 'state_holiday', 'sales']].groupby(['year', 'state_holiday']).sum().reset_index()
sns.barplot(x='year', y='sales', hue='state_holiday', data=aux2);

#sns.countplot(x=aux['state_holiday'])

In [ ]:
sns.countplot(x=aux['state_holiday'])

### H8 Lojas deveriam vendar mais ao longo dos anos

***FALSO***


In [ ]:
aux1 = df4[['year','sales']].groupby('year').sum().reset_index()

fig=plt.subplots(1,3,figsize=(21,7))

plt.subplot(1,3,1)
sns.barplot(x='year', y='sales', data = aux1);

plt.subplot(1,3,2)
sns.regplot(x='year', y='sales', data = aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

### H9 Lojas deveriam vender mais no segundo semestre do ano

***FALSO***

In [ ]:
aux1 = df4[['month','sales']].groupby('month').sum().reset_index()

fig=plt.subplots(1,3,figsize=(21,7))

plt.subplot(1,3,1)
sns.barplot(x='month', y='sales', data = aux1);

plt.subplot(1,3,2)
sns.regplot(x='month', y='sales', data = aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

### H11 Lojas deveriam vendar mais depois do dia 10 de cada mês

***VERDADEIRO***

In [ ]:
aux1 = df4[['day','sales']].groupby('day').sum().reset_index()

fig=plt.subplots(2,2,figsize=(21,14))

plt.subplot(2,2,1)
sns.barplot(x='day', y='sales', data = aux1);

plt.subplot(2,2,2)
sns.regplot(x='day', y='sales', data = aux1);

plt.subplot(2,2,4)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

aux1['before_after'] = aux1['day'].apply(lambda x: 'before_10_days' if x<= 10 else 'after_10_days')
aux2 = aux1[['before_after','sales']].groupby('before_after').sum().reset_index()

plt.subplot(2,2,3)
sns.barplot(x='before_after', y='sales', data = aux2);


### H12 Lojas deveriam vender menos aos finais de semana

***Verdadeiro***

In [ ]:
aux1 = df4[['day_of_week','sales']].groupby('day_of_week').sum().reset_index()

fig=plt.subplots(1,3,figsize=(21,7))

plt.subplot(1,3,1)
sns.barplot(x='day_of_week', y='sales', data = aux1);

plt.subplot(1,3,2)
sns.regplot(x='day_of_week', y='sales', data = aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

### H13 Lojas deveriam vender menos durante os feriados escolares.

***Verdadeiro***, Exceto no mes de agosto

In [ ]:
aux1 = df4[['school_holiday','sales']].groupby('school_holiday').sum().reset_index()
aux2 = df4[['month', 'school_holiday','sales']].groupby(['month', 'school_holiday']).sum().reset_index()

fig=plt.subplots(1,3,figsize=(21,7))

plt.subplot(1,3,1)
sns.barplot(x='school_holiday', y='sales', data = aux1);

plt.subplot(1,3,2)
sns.barplot(x='month', y='sales', hue='school_holiday', data = aux2);

'''plt.subplot(1,3,2)
sns.regplot(x='school_holiday', y='sales', data = aux1);'''

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

## 4.3 Analise multivariada
    Analise multivariada, separado pelas variaveis categoricas e núméricas

### Numericas

In [ ]:
fig = plt.figure(figsize=(14,7))
correlation = num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot=True);

### categorical

In [ ]:


a = cat_attributes

cramer_v(a['state_holiday'],a['store_type'])

a1 = cramer_v(a['state_holiday'],a['state_holiday'])
a2 = cramer_v(a['state_holiday'],a['store_type'])
a3 = cramer_v(a['state_holiday'],a['assortment'])

a4 = cramer_v(a['store_type'],a['state_holiday'])
a5 = cramer_v(a['store_type'],a['store_type'])
a6 = cramer_v(a['store_type'],a['assortment'])

a7 = cramer_v(a['assortment'],a['state_holiday'])
a8 = cramer_v(a['assortment'],a['store_type'])
a9 = cramer_v(a['assortment'],a['assortment'])

d = pd.DataFrame({'state_holiday': [a1,a2,a3],
              'store_type': [a4,a5,a6],
              'assortment': [a7,a8,a9]})

d = d.set_index(d.columns)
sns.heatmap(d, annot=True)

# 5.0 DATA PREPARATION
    Preparação dos dados. Aqui é tratado as colunas, normalizando-as quando necessário

In [ ]:
df5 = df4.copy()
a = df5.select_dtypes(include=['int32', 'int64', 'float64'])

## 5.1 Normalização

### Nenhuma variavel com distribuição normal
    A normalização funciona bem variaveis que possuem distribuição normal. Como visto na analise univariavel nenhum possue esse tipo de distribuição, portanto não foi aplicado



## 5.2 Rescaling

    Como não temos nenhuma variavel categórica, o método de reescaling foi aplicado. MinMaxScaler foi utilizado para colunas que não apresentam Outliers; quando existe outliers foi aplicado o RobustScaler

In [ ]:
rs = RobustScaler()
mms = MinMaxScaler()

#sns.boxplot(df5['competition_distance'])
df5['competition_distance']=rs.fit_transform(a[['competition_distance']].values)
pickle.dump(rs, open(os.path.join(PARA_DIR,'competition_distance_scaler.pkl'), 'wb'))

#sns.boxplot(df5['competition_time_month'])
df5['competition_time_month']=rs.fit_transform(a[['competition_time_month']].values)
pickle.dump(rs, open(os.path.join(PARA_DIR,'competition_time_month_scaler.pkl'), 'wb'))


#sns.boxplot(df5['promo_time_week'])
df5['promo_time_week']=mms.fit_transform(a[['promo_time_week']].values)
pickle.dump(mms, open(os.path.join(PARA_DIR,'promo_time_week_scaler.pkl'), 'wb'))

#sns.boxplot(df5['year'])
df5['year']=mms.fit_transform(a[['year']].values)
pickle.dump(mms, open(os.path.join(PARA_DIR,'year_scaler.pkl'), 'wb'))

## 5.3 Transformação

    Nas variaveis categoricas foi aplicado técnicas de transformação, como o onehotencoding, label encoding e ordinal encoding

In [ ]:
#state holiday - onehotencoding
df5 = pd.get_dummies(df5,prefix=['state_holiday'], columns=['state_holiday'],dtype=float)

#store_type  Label encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])
pickle.dump(le, open(os.path.join(PARA_DIR,'store_type_scaler.pkl'), 'wb'))

#store_type  ordinal encoding
assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
df5['assortment'] = df5['assortment'].map(assortment_dict)


    Para variaveis ciclicas (ano, meses, semanas e dias) foi utilizado uma técnica de escrevelas em sua decomposição em seno e cosseno

In [ ]:
#Transformação em variaveis ciclicas

df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x*(2*np.pi/12)))
df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x*(2*np.pi/12)))

df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x*(2*np.pi/30)))
df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x*(2*np.pi/30)))

df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x*(2*np.pi/52)))
df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x*(2*np.pi/52)))

df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x*(2*np.pi/7)))
df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x*(2*np.pi/7)))

# 6.0 FEATURE SELECTION
    Nessa seção vamos selecionar as varivaveis de interesse para o modelo. Algumas variaveis que foram utilizadas para definir outras serão automaticamente descartadas (para evitar dependencia linear no modelo). As demais serão escolhidas pelo método do algoritmo Boruta.

## 6.1 Preparação dos dados

In [ ]:
df6 = df5.copy()
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week']
df6 = df6.drop(cols_drop, axis=1)

    Separação de treino e teste para utilizar no algoritmo Boruta

In [ ]:
X_train = df6[df6['date']<'2015-06-19']
y_train = X_train['sales']

X_teste = df6[df6['date']>='2015-06-19']
y_teste = X_teste['sales']


## 6.2 Aplicação Boruta

In [ ]:
X_train_n = X_train.drop(['date','sales'], axis=1).values
y_train_n = y_train.values.ravel()

rf = RandomForestRegressor(n_jobs=-1)

boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=43).fit(X_train_n, y_train_n)

    Armazenamento das variaveis classificadas como boa pelo boruta

In [ ]:
cols_selected = boruta.support_.tolist()

X_train_fs = X_train.drop(['date', 'sales'], axis=1)

cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

cols_not_selected_boruta = list(np.setdiff1d(X_train_fs.columns, cols_selected_boruta))

In [ ]:
cols_selected_boruta_bau = ['store',
                            'promo',
                            'store_type',
                            'assortment',
                            'competition_distance',
                            'competition_open_since_month',
                            'competition_open_since_year',
                            'promo2',
                            'promo2_since_week',
                            'promo2_since_year',
                            'competition_time_month',
                            'promo_time_week',
                            'month_sin',
                            'month_cos',
                            'day_sin',
                            'day_cos',
                            'day_of_week_sin',
                            'day_of_week_cos',
                            'week_of_year_cos',
                            'week_of_year_sin']

feat_to_add = ['date', 'sales']

cols_selected_boruta_full = cols_selected_boruta_bau.copy()
cols_selected_boruta_full.extend(feat_to_add)


# 7 MACHINE LEARGING MODEL

## 7.1 Aplicação do modelo

In [ ]:
x_train = X_train[cols_selected_boruta_bau]
x_teste = X_teste[cols_selected_boruta_bau]
x_training = X_train[cols_selected_boruta_full]


## Media

In [ ]:
aux1 = x_teste.copy()
aux1['sales'] = y_teste.copy()

#predict
aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'predictions'})
aux1 = pd.merge(aux1, aux2, how='left', on='store')
yhat_baseline = aux1['predictions']

baseline_result = ml_error('Average Model', y_teste, yhat_baseline)
baseline_result

## Linear regression

In [ ]:
lr = LinearRegression().fit(x_train, y_train)

yhat_lr = lr.predict(x_teste)

lr_result = ml_error( 'LinearRegression', y_teste, yhat_lr)
lr_result

## Linear regression - cv

In [ ]:
#lr = LinearRegression()
lr_result_cv = cross_validation(x_training, 5, 'Linear Regression', lr, verbose=False)
lr_result_cv

## Linear Regression regularizado LASSO

In [ ]:
lrr = Lasso().fit(x_train, y_train)

yhat_lrr = lrr.predict(x_teste)

lrr_result = ml_error( 'LinearRegression - LASSO', y_teste, yhat_lrr)
lrr_result

## Linear Regression reg - cv

In [ ]:
lrr_result_cv = cross_validation(x_training, 5, 'Lasso', lrr, verbose=True)
lrr_result_cv

## Random forest regressor


In [ ]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=43).fit(x_train, y_train)

yhat_rf = rf.predict(x_teste)

rf_result = ml_error( 'RFR', y_teste, yhat_rf)
rf_result

## Random forest reg - cv


In [ ]:
rf_result_cv = cross_validation(x_training, 5, 'Random Forest Regressor', rf, verbose=True)
rf_result_cv

## XGBoost Regressor

In [ ]:
model_xgb = xgb.XGBRegressor(objective='reg:squarederror').fit(x_train, y_train)

yhat_xgb = model_xgb.predict(x_teste)

xgb_result = ml_error( 'xgboost', y_teste, yhat_xgb)
xgb_result

## XGBoost Reg - cv

In [ ]:
xgb_result_cv = cross_validation(x_training, 5, 'XGB Regressor', model_xgb, verbose=True)
xgb_result_cv

## Comparação de performance

In [ ]:
modelling_result = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result])
modelling_result.sort_values('RMSE')

## CV Performance

In [ ]:
modelling_result_cv = pd.concat([lr_result_cv, lrr_result_cv, rf_result_cv, xgb_result_cv])
modelling_result_cv.sort_values('RMSE CV')

# 8.0 HYPERPARAMETER FINE TUNING

## 8.1 Random Search

In [ ]:
import random

In [ ]:
param = {'eta': [0.1, 0.2, 0.3],
         'max_depth': [4, 6, 8],
         'subsample': [1, 0.8, 0.6],
         'colsample_bytree': [1, 0.8, 0.6],
         'min_child_weight': [1, 6, 11]}

MAX_EVAL = 10

In [ ]:
final_result = pd.DataFrame()

for i in range (MAX_EVAL):
    
    hp = {k: random.sample(v, 1)[0] for k, v in param.items()}
    print(hp)
    
    model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                eta = hp['eta'],
                                max_depth = hp['max_depth'],
                                subsample = hp['subsample'],
                                colsample_bytree=hp['colsample_bytree'],
                                min_child_weight=hp['min_child_weight'])
    

    result = cross_validation( x_training, 5, 'XGB', model_xgb, verbose=True)
    final_result = pd.concat([final_result, result])

final_result



## Final model

In [ ]:
param_tuned = {'eta': 0.3,
               'max_depth': 8,
               'subsample': 1,
               'colsample_bytree': 1,
               'min_child_weight': 1}


In [ ]:
model_xgb_tuned = xgb.XGBRegressor(objective='reg:squarederror',
                                   n_estimators = 3000,
                                   eta = param_tuned['eta'],
                                   max_depth = param_tuned['max_depth'],
                                   subsample = param_tuned['subsample'],
                                   colsample_bytree=param_tuned['colsample_bytree'],
                                   min_child_weight=param_tuned['min_child_weight']).fit(x_train, y_train)
    
yhat_tuned = model_xgb_tuned.predict(x_teste)

result_tuned = ml_error('XGB', y_teste, yhat_tuned)
result_tuned

# 9.0 TRADUÇÃO E INTERPRETAÇÃO DO ERRO

In [ ]:
df9 = X_teste[cols_selected_boruta_full].copy()
df9['predictions'] = yhat_tuned

## Business performance

In [ ]:
df9_1 = df9[['store', 'predictions']].groupby('store').sum().reset_index()

df9_aux1 = df9[['store', 'sales', 'predictions']].groupby('store').apply(lambda x: mean_absolute_error(x['sales'], x['predictions'])).reset_index().rename(columns={0: 'MAE'})
df9_aux2 = df9[['store', 'sales', 'predictions']].groupby('store').apply(lambda x: mean_absolute_percentage_error(x['sales'], x['predictions'])).reset_index().rename(columns={0: 'MAPE'})
df9_aux3 = pd.merge(df9_aux1, df9_aux2, how='inner', on='store')

df9_2 = pd.merge(df9_1, df9_aux3, how='inner', on='store')
df9_2

#Pior cenario
df9_2['worst_scenario'] = df9_2['predictions']-df9_2['MAE'] 
df9_2['best_scenario'] = df9_2['predictions']+df9_2['MAE'] 

df9_2 = df9_2[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE']]
df9_2

In [ ]:
df9_2.sort_values('MAPE', ascending=False)

In [ ]:
sns.scatterplot(x='store', y='MAPE', data=df9_2)

In [ ]:
df9_3 = df9_2[['predictions', 'worst_scenario', 'best_scenario']].apply(lambda x: np.sum(x), axis=0).reset_index().rename(columns={'index': 'Scenarios', 0: 'Values'})
df9_3['Values'] = df9_3['Values'].map('${:,.2f}'.format)
df9_3

In [ ]:
df9['error'] = df9['sales']- df9['predictions']
df9['error_rate'] = df9['predictions']/df9['sales']

fig = plt.subplots(2,2,figsize=(21,14))

plt.subplot(2,2,1)
sns.lineplot(x='date', y='sales', data=df9, label='SALES')
sns.lineplot(x='date', y='predictions', data=df9, label='PREDICTIONS')

plt.subplot(2,2,2)
sns.lineplot(x='date', y='error_rate', data=df9)


plt.subplot(2,2,3)
sns.histplot(df9['error'], kde=True, stat="density");

plt.subplot(2,2,4)
sns.scatterplot(x=df9['predictions'], y=df9['error'])




# 10.0 DEPLOY

In [ ]:

pickle.dump(model_xgb_tuned, open(os.path.join(PARA_DIR,'model_rossman.pkl'), 'wb'))


In [ ]:
import pickle
import inflection 
import pandas as pd
import numpy as np
import math


class Rossmann (object):
    
    def __init__(self):
        self.home_path = '/home/mayconr/repos/ComunidadeDS/dsproducao/rossmann_project/'
        self.competition_time_month_scaler  = pickle.load(open(self.home_path + 'parameter/competition_time_month_scaler.pkl', 'rb'))
        self.competition_distance_scaler    = pickle.load(open(self.home_path + 'parameter/competition_distance_scaler.pkl', 'rb'))
        self.promo_time_week_scaler         = pickle.load(open(self.home_path + 'parameter/promo_time_week_scaler.pkl', 'rb'))
        self.store_type_scaler              = pickle.load(open(self.home_path + 'parameter/store_type_scaler.pkl', 'rb'))
        self.year_scaler                    = pickle.load(open(self.home_path + 'parameter/year_scaler.pkl', 'rb'))
        
    
    def data_cleaning(self, df1):
     
        cols_old = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
                    'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

        snakecase = lambda x: inflection.underscore(x)
        cols_new = list(map(snakecase, cols_old))

        #rename
        df1.columns = cols_new

        df1['date'] = pd.to_datetime(df1['date'])
        
        # competition_distance
        df1['competition_distance'].fillna(200000.0, inplace=True)

        # competition_open_since_month
        df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

        # competition_open_since_year
        df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

        # promo2_since_week
        df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

        # promo2_since_year
        df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

        # promo_interval
        df1['promo_interval'].fillna(0, inplace=True)


        month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
        df1['month_map'] = df1['date'].dt.month.map(month_map)
        df1['is_promo'] = df1.apply(is_promo, axis=1)

        df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
        df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
        df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
        df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)
        
        return df1
    
    
    def feature_engineering(self, df2):
        #Year
        df2['year'] = df2['date'].dt.year

        #Month
        df2['month'] = df2['date'].dt.month

        #Day
        df2['day'] = df2['date'].dt.day

        #Week of Year
        df2['week_of_year'] = df2['date'].dt.isocalendar().week

        #Year week
        df2['year_week'] = df2['date'].dt.strftime('%Y-%W')


        #competition since
        df2['competition_since'] = df2.apply(lambda x: datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1),axis=1)
        df2['competition_time_month'] = ((df2['date']-df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

        #Promo since
        df2['promo_since'] = df2['promo2_since_year'].astype(str)+'-'+df2['promo2_since_week'].astype(str)
        df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.strptime(x+'-1','%Y-%W-%w')-timedelta(days=7))
        df2['promo_time_week'] = ((df2['date']-df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

        #assortment
        df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x=='a' else 'extra' if x=='b' else 'extended')

        #state holiday
        df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x=='a' else 'easter_holiday' if x=='b' else 'christmas' if x == 'c' else 'regular_day')

        ## 3.1 Filtragem das linhas

        df2 = df2[df2['open'] != 0]
        ## 3.2 Seleção das colunas

        cols_drop = ['open', 'promo_interval', 'month_map']
        df2 = df2.drop(cols_drop, axis=1)
        
        return df2
    
    def data_preparation(self,df5):
    

        a = df5.select_dtypes(include=['int32', 'int64', 'float64'])

        df5['competition_distance']= self.competition_distance_scaler.fit_transform(a[['competition_distance']].values)

        df5['competition_time_month']=self.competition_time_month_scaler.fit_transform(a[['competition_time_month']].values)
        
        df5['promo_time_week']=self.promo_time_week_scaler.fit_transform(a[['promo_time_week']].values)

        df5['year']=self.year_scaler.fit_transform(a[['year']].values)


        #state holiday - onehotencoding
        df5 = pd.get_dummies(df5,prefix=['state_holiday'], columns=['state_holiday'],dtype=float)

        df5['store_type'] = self.store_type_scaler.fit_transform(df5['store_type'])


        #store_type  ordinal encoding
        assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
        df5['assortment'] = df5['assortment'].map(assortment_dict)

        df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x*(2*np.pi/12)))
        df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x*(2*np.pi/12)))

        df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x*(2*np.pi/30)))
        df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x*(2*np.pi/30)))

        df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x*(2*np.pi/52)))
        df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x*(2*np.pi/52)))

        df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x*(2*np.pi/7)))
        df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x*(2*np.pi/7)))
        
        cols_selected = ['store','promo','store_type','assortment','competition_distance',
                         'competition_open_since_month','competition_open_since_year','promo2',
                         'promo2_since_week','promo2_since_year','competition_time_month',
                         'promo_time_week','month_sin','month_cos','day_sin','day_cos',
                         'day_of_week_sin','day_of_week_cos','week_of_year_cos','week_of_year_sin']

        
        return df5[cols_selected]
    
    def get_prediction(self, model, original_data, test_data):
        pred=model.predict(test_data)
        original_data['prediction'] = pred
        return original_data.to_json(orient='records', date_format='iso')
    

In [ ]:
import pandas as pd
import pickle
from flask import Flask, request, Response
from rossmann.Rossmann import Rossmann

#load model

model = pickle.load(open('/home/mayconr/repos/ComunidadeDS/dsproducao/rossmann_project/model/model_rossman.pkl', 'rb'))

app = Flask(__name__)

@app.route('/rossmann/predict', methods=['POST'])
def rossmann_predict():
    test_json = request.get_json()
    
    if test_json: #there is data
        if isinstance(test_json, dict): #unique example
            teste_raw = pd.DataFrame(test_json, index=[0])
        else: #Multiple example
            teste_raw = pd.DataFrame(test_json, columns= test_json[0].keys())
            
        # instantiate rossmann class
        pipeline = Rossmann()
        
        #data cleaning
        df1 = pipeline.data_cleaning(teste_raw)
        
        #data engineering
        df2 = pipeline.feature_engineering(df1)
        
        #data preparation
        df3 = pipeline.data_preparation(df2)
        
        #prediction
        df_response = pipeline.get_prediction(model, teste_raw, df3)
        
        return df_response
        
        
    else:
        return Response('{}', status=200, mimetype='application/json')

if __name__ == '__main__':
    app.run('0.0.0.0')

In [206]:
df10 = pd.read_csv('/home/mayconr/repos/ComunidadeDS/dsproducao/rossmann_project/data/test.csv')

df_test = pd.merge(df10, df_store_raw, how= 'left', on= 'Store')

df_test = df_test[df_test['Store'].isin([22, 23, 24, 25])]

df_test = df_test[df_test['Open']!=0]
df_test = df_test[~df_test['Open'].isnull()]

df_test = df_test.drop('Id', axis=1)

data = df_test.to_dict(orient='records')

In [200]:
import requests
import json

In [207]:

url = 'http://0.0.0.0:5000/rossmann/predict'

header = {'Content-type': 'application/json'} 

#data = data



r = requests.post(url, data=json.dumps(data), headers=header)
print('Status Code{}'.format(r.status_code))

Status Code200


In [208]:
d1 = pd.DataFrame(r.json(), columns=r.json()[0].keys())
d1

,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,...,year,month,day,week_of_year,year_week,competition_since,competition_time_month,promo_since,promo_time_week,prediction
0,22,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,a,basic,1040.0,...,2015,9,17,38,2015-37,2015-09-01T00:00:00.000,0,2012-05-21T00:00:00.000,173,8796.821289
1,23,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,d,basic,4060.0,...,2015,9,17,38,2015-37,2005-08-01T00:00:00.000,123,2015-09-14T00:00:00.000,0,10667.041016
2,24,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,a,extended,4590.0,...,2015,9,17,38,2015-37,2000-03-01T00:00:00.000,189,2011-09-26T00:00:00.000,207,8865.985352
3,25,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,c,basic,430.0,...,2015,9,17,38,2015-37,2003-04-01T00:00:00.000,151,2015-09-14T00:00:00.000,0,13095.437500
4,22,3,2015-09-16T00:00:00.000,1.0,1,regular_day,0,a,basic,1040.0,...,2015,9,16,38,2015-37,2015-09-01T00:00:00.000,0,2012-05-21T00:00:00.000,173,8490.807617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,25,1,2015-08-03T00:00:00.000,1.0,1,regular_day,1,c,basic,430.0,...,2015,8,3,32,2015-31,2003-04-01T00:00:00.000,150,2015-08-03T00:00:00.000,0,14403.689453
159,22,6,2015-08-01T00:00:00.000,1.0,0,regular_day,0,a,basic,1040.0,...,2015,8,1,31,2015-30,2015-08-01T00:00:00.000,0,2012-05-21T00:00:00.000,166,5899.893066
160,23,6,2015-08-01T00:00:00.000,1.0,0,regular_day,0,d,basic,4060.0,...,2015,8,1,31,2015-30,2005-08-01T00:00:00.000,121,2015-07-27T00:00:00.000,0,7222.688477
161,24,6,2015-08-01T00:00:00.000,1.0,0,regular_day,0,a,extended,4590.0,...,2015,8,1,31,2015-30,2000-03-01T00:00:00.000,187,2011-09-26T00:00:00.000,200,6072.159668


In [209]:
d2 =d1[['store', 'prediction']].groupby('store').sum().reset_index()

for i in range(len(d2)):
    print('Sotre number {} will sell ${:,.2f} in the next 6 weeks'. format(d2.loc[i,'store'], d2.loc[i,'prediction']))


Sotre number 22 will sell $248,822.87 in the next 6 weeks
Sotre number 23 will sell $383,744.36 in the next 6 weeks
Sotre number 24 will sell $342,771.86 in the next 6 weeks
Sotre number 25 will sell $524,725.90 in the next 6 weeks
